# Working with Data

Here is an example of loading & plotting data from the archive

In [3]:
import intake
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:

tmp_cat = {
    
    'AMB_CCBV03': {
        'args': {
            'fastzarr': True,
            'urlpath': '/home/lhs18285/mast-data/mast/AMB_PLASMA_CURRENT.zarr'
            
        },
        'description': 'AMC_PLASMA_CURRENT',
        'driver': 'intake_xarray_datatree.intake_xarray_datatree.DataTreeSource'
    }
}
# %%writefile clustering_details.yaml

In [9]:
catalog = intake.open_catalog(Path('~/mast-data/mast/catalog.yml').expanduser())
# Path(catalog.path).parent / 'AMC_PLASMA_CURRENT.zarr'
catalog.AMB_CCBV03

AMB_CCBV03:
  args:
    fastzarr: true
    urlpath: /home/lhs18285/mast-data/mast/AMB_CCBV03.zarr
  description: CCBV03
  driver: intake_xarray_datatree.intake_xarray_datatree.DataTreeSource
  metadata:
    catalog_dir: /home/lhs18285/mast-data/mast/
    description: ''
    label: CCBV03
    rank: 1
    shape:
    - 23000
    time_index: 0
    units: Tesla


In [14]:

dataset = data['AMC_PLASMA_CURRENT']
# Choose only relevant shots
dataset = dataset.filter(lambda x: x.name in shot_ids)
dataset = dataset.sel(time=slice(0, .5))

for shot_id, shot in dataset.items():
    data = shot['data']
    time = shot['time']
    plt.plot(time, data, label=f'Shot {shot_id}') 

plt.ylabel(f'{shot.label} ({shot.units})')
plt.xlabel('Time')
plt.legend()

GroupNotFoundError: group not found at path ''

Another example with the same data but with multi dimensional data this time. Use EFM PSI which should be an equillibrium reconstruction.

In [12]:
# Query for signal data, filter only names containing 'acd'
qsignal = (
    session.query(signals)
    .filter(signals.c.name.contains('EFM_PSI(R,Z)'))
)

result = pd.read_sql(qsignal.statement, con=engine.connect())
result

,signal_id,name,units,rank,dim_1_label,dim_2_label,dim_3_label,uri,description,signal_type,quality,doi,camera_metadata,camera
0,882,"EFM_PSI(R,Z)",Wb/rad,3,None,None,None,/home/lhs18285/git/fair-mast/data/mast/zarr/EF...,"psi(r,z)",Analysed,Not Checked,,None,None


In [8]:
# Read data 
data = {row['name']: open_datatree(row.uri, engine='zarr') for index, row in result[['name', 'uri']].iterrows()}

# Choose only relevant shots
dataset = data['EFM_PSI(R,Z)']
dataset = dataset.filter(lambda x: x.name in shot_ids)
dataset = dataset.isel(time=50)

n_shots = len(dataset)
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
axes = axes.flatten()

for index, (shot_id, shot) in enumerate(dataset.items()):
    data = shot['data']
    time = shot['time']
    axes[index].matshow(data, cmap='plasma')
    axes[index].set_xticks([], [])
    axes[index].set_yticks([], [])
    axes[index].set_title(f'Shot {shot_id}')
plt.suptitle(f'EFM_PSI(R,Z) ({shot.attrs["units"]})')
plt.tight_layout()

IndexError: index 50 is out of bounds for axis 0 with size 16